# Service-Aware Google Colab Notebook

In this notebook, we load and pre-process the data collected from the network traffic scenarios. Moreover, we define, train and evaluate an LSTM model to fit the data optimally.
<br>
<br>

### Upload collected data

In [6]:
from google.colab import files
uploaded = files.upload()

Saving train_data.csv to train_data.csv
Saving data2.csv to data2.csv


### Mount to Google Drive

We mount the notebook to google drive to save our final model.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Import necessary libraries

In [3]:
import os
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense
from os import walk
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

from random import random
from random import randint
from numpy import array
from numpy import zeros
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed

from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import RepeatVector
from keras.layers import Bidirectional
from keras.layers import GRU

from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# Pre-processing

Start with pre-processing the augmentation data.

In [7]:
train_df = pd.read_csv('train_data.csv')
train_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,27254.0,0.0,0.0,0.0,24751.0,0.0,0.0,0.0,0.0,0.02023966779233446,0.01527286153105846,None,11.0,15.0,15.0
1,6764.0,0.0,0.0,0.0,22407.0,0.0,0.0,0.0,0.0,0.026870355690138334,0.018230975024189792,None,7.0,9.0,12.0
2,10535.0,0.0,0.0,0.0,26582.0,0.0,0.0,0.0,0.0,0.024350793637440978,0.017787373553197528,None,10.0,14.0,14.0
3,10196.0,0.0,0.0,0.0,24831.0,0.0,0.0,0.0,0.0,0.031298670300758925,0.01725335284054473,None,3.0,8.0,13.0
4,13098.0,0.0,0.0,0.0,24589.0,0.0,0.0,0.0,24362.0,0.03184602244585295,0.016259092906849622,0.006921501128689515,8.0,8.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35095,0.0,0.0,0.0,11595.0,0.0,0.0,0.0,0.0,0.0,None,0.017877045915793758,None,8.0,8.0,14.0
35096,0.0,0.0,0.0,18524.0,0.0,0.0,0.0,0.0,0.0,None,0.019695794167362186,None,8.0,7.0,13.0
35097,0.0,0.0,0.0,6048.0,0.0,0.0,0.0,0.0,0.0,None,0.02060436174599804,None,9.0,9.0,13.0
35098,0.0,0.0,0.0,21424.0,0.0,0.0,0.0,0.0,0.0,None,0.016930942004330388,None,6.0,7.0,12.0


In [8]:
test_df = pd.read_csv('data2.csv')
test_df = test_df.drop(['Time'],axis=1)
test_df = test_df.drop(['empty'],axis=1)
test_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,27058,0,0,0,26362,0,0,0,0,0.020392699062500002,0.016950569033898303,None,13,15,15
1,6653,0,0,0,23589,0,0,0,0,0.026422801260273972,0.017882072375,None,8,8,13
2,10327,0,0,0,29718,0,0,0,0,0.02791389094285714,0.01671926272067039,None,11,14,15
3,10905,0,0,0,26210,0,0,0,0,0.028728796345588233,0.016630520718487394,None,3,8,13
4,12492,0,0,0,26210,0,0,0,24484,0.028863872136904764,0.016647158656565655,0.005963571786585364,7,8,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0,0,0,13414,0,0,0,0,0,None,0.017832543045126697,None,7,8,13
113,0,0,0,16824,0,0,0,0,0,None,0.01789545736039678,None,7,8,13
114,0,0,0,6506,0,0,0,0,0,None,0.01799716347998539,None,8,8,12
115,0,0,0,20683,0,0,0,0,0,None,0.017994438103654135,None,7,8,12


Replace None Values

In [12]:
train_df = train_df.replace('None', 0)
test_df = test_df.replace('None', 0)

### Define Pre-processing Functions

In [9]:
def extract_X_y_from_file(array, X_num, y_num):
    """ 
    Function that pre-process the data with a sliding-window supervised 
    learning approach extracting X and y

    - X_num: Input Window Length (N)
    - y_num: Number of prediction time-steps (M). We calculate the y as a mean 
              of the next M time-slots.
    """

    X = []
    y = []
    
    iterations = len(array)
    for i in range(iterations):
        ### check if we reach to end
        if i+X_num + y_num > len(array):
            break
            
        ### calculate X,y
        X_temp = array[i: i+X_num]
        y_temp = array[i+X_num: i+X_num + y_num]
        
        ### format y use mean
        new_y_temp = []
        # for every column
        for j in range(15):
            j_temp = []
            # for every row
            for i in range(y_num):
                row = y_temp[i][j]                

                j_temp.append(row)

            # obtain mean
            new_y_temp.append(np.mean(j_temp))
                        
        
        # append
        X.append(X_temp)
        y.append(new_y_temp)        
        
    return X,y

In [10]:
def create_data(df, X_num = 30, y_num = 5):
    """
    Function that reads all data csvs and calls the above mentioned function to
    extract X and y
    """
    all_X = []
    all_y = []

    # convert to list
    array = df.values.tolist()

    # create X and y
    X,y = extract_X_y_from_file(array, X_num, y_num)

    # add to final data
    all_X = all_X + X
    all_y = all_y + y

    return all_X, all_y

Convert object columns to float

In [13]:
train_df['UE1-Jitter'] = train_df['UE1-Jitter'].astype(float)
train_df['UE2-Jitter'] = train_df['UE2-Jitter'].astype(float)
train_df['UE3-Jitter'] = train_df['UE3-Jitter'].astype(float)
train_df.dtypes

UE1: web-rtc       float64
UE1: sipp          float64
UE1: web-server    float64
UE2: web-rtc       float64
UE2: sipp          float64
UE2: web-server    float64
UE3: web-rtc       float64
UE3: sipp          float64
UE3: web-server    float64
UE1-Jitter         float64
UE2-Jitter         float64
UE3-Jitter         float64
UE1-CQI            float64
UE2-CQI            float64
UE3-CQI            float64
dtype: object

In [14]:
test_df['UE1-Jitter'] = test_df['UE1-Jitter'].astype(float)
test_df['UE2-Jitter'] = test_df['UE2-Jitter'].astype(float)
test_df['UE3-Jitter'] = test_df['UE3-Jitter'].astype(float)
test_df.dtypes

UE1: web-rtc         int64
UE1: sipp            int64
UE1: web-server      int64
UE2: web-rtc         int64
UE2: sipp            int64
UE2: web-server      int64
UE3: web-rtc         int64
UE3: sipp            int64
UE3: web-server      int64
UE1-Jitter         float64
UE2-Jitter         float64
UE3-Jitter         float64
UE1-CQI              int64
UE2-CQI              int64
UE3-CQI              int64
dtype: object

# Normalize Data

In [15]:
from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
train_df_normalized = pd.DataFrame(scaler.fit_transform(train_df), columns=train_df.columns)
test_df_normalized = pd.DataFrame(scaler.fit_transform(test_df), columns=test_df.columns)  



In [16]:
train_df_normalized

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,0.969859,0.0,0.0,0.000000,0.732864,0.0,0.0,0.0,0.000000,0.413713,0.627131,0.000000,0.714286,1.000000,1.000
1,0.240703,0.0,0.0,0.000000,0.663459,0.0,0.0,0.0,0.000000,0.549249,0.748597,0.000000,0.428571,0.571429,0.625
2,0.374898,0.0,0.0,0.000000,0.787078,0.0,0.0,0.0,0.000000,0.497747,0.730382,0.000000,0.642857,0.928571,0.875
3,0.362834,0.0,0.0,0.000000,0.735232,0.0,0.0,0.0,0.000000,0.639767,0.708454,0.000000,0.142857,0.500000,0.750
4,0.466104,0.0,0.0,0.000000,0.728067,0.0,0.0,0.0,0.831808,0.650955,0.667628,0.470044,0.500000,0.500000,0.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35095,0.000000,0.0,0.0,0.230073,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.734064,0.000000,0.500000,0.500000,0.875
35096,0.000000,0.0,0.0,0.367562,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.808745,0.000000,0.500000,0.428571,0.750
35097,0.000000,0.0,0.0,0.120007,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.846052,0.000000,0.571429,0.571429,0.750
35098,0.000000,0.0,0.0,0.425105,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.695215,0.000000,0.357143,0.428571,0.625


In [17]:
test_df_normalized

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,1.000000,0.0,0.0,0.000000,0.886267,0.0,0.0,0.0,0.000000,0.546560,0.937291,0.000000,1.0,1.000000,1.00
1,0.245879,0.0,0.0,0.000000,0.793041,0.0,0.0,0.0,0.000000,0.708177,0.988799,0.000000,0.5,0.363636,0.50
2,0.381662,0.0,0.0,0.000000,0.999092,0.0,0.0,0.0,0.000000,0.748140,0.924500,0.000000,0.8,0.909091,1.00
3,0.403023,0.0,0.0,0.000000,0.881156,0.0,0.0,0.0,0.000000,0.769981,0.919593,0.000000,0.0,0.363636,0.50
4,0.461675,0.0,0.0,0.000000,0.881156,0.0,0.0,0.0,0.940499,0.773602,0.920513,0.566547,0.4,0.363636,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0.000000,0.0,0.0,0.284768,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.986060,0.000000,0.4,0.363636,0.50
113,0.000000,0.0,0.0,0.357160,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.989539,0.000000,0.4,0.363636,0.50
114,0.000000,0.0,0.0,0.138117,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.995163,0.000000,0.5,0.363636,0.25
115,0.000000,0.0,0.0,0.439083,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.995012,0.000000,0.4,0.363636,0.25


# Sliding-Window Approach 
Create windows of data with the sliding-window technique to construct a supervised learning problem.

In [18]:
all_X, all_y = create_data(df=train_df_normalized)

In [19]:
len(all_X)

35066

In [20]:
len(all_y)

35066

In [21]:
X_data = np.array(all_X)
X_data.shape

(35066, 30, 15)

In [22]:
y_data = np.array(all_y)
print(y_data.shape)

(35066, 15)


Pre-process the basic non-augmented scenario to use as validation data.

In [23]:
val_X = []
val_y = []

val_X, val_y = create_data(df=test_df_normalized)

X_val = np.array(val_X)
print(X_val.shape)

y_val = np.array(val_y)
print(y_val.shape)

(83, 30, 15)
(83, 15)


In [24]:
print(X_data.shape)
print(y_data.shape)
print(X_val.shape)
print(y_val.shape)

(35066, 30, 15)
(35066, 15)
(83, 30, 15)
(83, 15)


# CNN-LSTM Model Implementation

In [28]:
# define the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(30,15)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(1))
model.add(LSTM(25, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(15))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 29, 64)            1984      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 14, 64)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 896)               0         
                                                                 
 repeat_vector_1 (RepeatVect  (None, 1, 896)           0         
 or)                                                             
                                                                 
 lstm_2 (LSTM)               (None, 1, 25)             92200     
                                                                 
 lstm_3 (LSTM)               (None, 25)               

Define a callback function to save on every iteration the best optimal trained model, based on validation accuracy. Then fit the model to the data.

In [29]:
from keras.callbacks import ModelCheckpoint
filepath="/content/gdrive/MyDrive/app_aware_CNN-LSTM_15cols.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [82]:
# save scaler
import joblib
scaler_filename = F"/content/gdrive/MyDrive/scaler_app_aware_CNN-LSTM.save"
joblib.dump(scaler, scaler_filename)

['/content/gdrive/MyDrive/scaler_app_aware_CNN-LSTM.save']

In [30]:
model.fit(X_data,y_data, validation_data=(X_val,y_val),epochs=100, callbacks=callbacks_list)
#model.fit(X_data,y_data,epochs=300)

Epoch 1/100
1094/1096 [============================>.] - ETA: 0s - loss: 0.0099
Epoch 1: val_loss improved from inf to 0.01941, saving model to /content/gdrive/MyDrive/app_aware_CNN-LSTM_15cols.best.hdf5
1096/1096 [==============================] - 13s 8ms/step - loss: 0.0099 - val_loss: 0.0194
Epoch 2/100
1095/1096 [============================>.] - ETA: 0s - loss: 0.0010
Epoch 2: val_loss improved from 0.01941 to 0.01798, saving model to /content/gdrive/MyDrive/app_aware_CNN-LSTM_15cols.best.hdf5
1096/1096 [==============================] - 8s 7ms/step - loss: 0.0010 - val_loss: 0.0180
Epoch 3/100
1093/1096 [============================>.] - ETA: 0s - loss: 8.6599e-04
Epoch 3: val_loss improved from 0.01798 to 0.01649, saving model to /content/gdrive/MyDrive/app_aware_CNN-LSTM_15cols.best.hdf5
1096/1096 [==============================] - 8s 7ms/step - loss: 8.6594e-04 - val_loss: 0.0165
Epoch 4/100
1095/1096 [============================>.] - ETA: 0s - loss: 8.0044e-04
Epoch 4: val_l

Now the model is trained. Make a prediction test ( with 1 value, the X[0] ) to present the format of the input and the format of the predictions.

In [31]:
test = X_data[0].reshape(1,30,15)
test.shape

(1, 30, 15)

In [32]:
test

array([[[0.96985872, 0.        , 0.        , 0.        , 0.73286353,
         0.        , 0.        , 0.        , 0.        , 0.41371292,
         0.62713122, 0.        , 0.71428571, 1.        , 1.        ],
        [0.24070318, 0.        , 0.        , 0.        , 0.66345898,
         0.        , 0.        , 0.        , 0.        , 0.5492488 ,
         0.7485967 , 0.        , 0.42857143, 0.57142857, 0.625     ],
        [0.37489769, 0.        , 0.        , 0.        , 0.78707844,
         0.        , 0.        , 0.        , 0.        , 0.4977472 ,
         0.73038162, 0.        , 0.64285714, 0.92857143, 0.875     ],
        [0.36283406, 0.        , 0.        , 0.        , 0.73523229,
         0.        , 0.        , 0.        , 0.        , 0.63976664,
         0.70845377, 0.        , 0.14285714, 0.5       , 0.75      ],
        [0.46610441, 0.        , 0.        , 0.        , 0.7280668 ,
         0.        , 0.        , 0.        , 0.83180825, 0.6509549 ,
         0.66762766, 0.4700443

In [33]:
yhat = model.predict(test)
yhat

array([[ 2.4270178e-01,  3.9928069e-05,  1.7467871e-02, -2.4060756e-03,
         7.7999896e-01, -1.5668722e-04,  6.7217463e-01,  9.9189048e-09,
        -8.9511275e-04,  7.6353908e-01,  6.6489756e-01,  6.4964604e-01,
         4.0254569e-01,  5.1549751e-01,  7.6424193e-01]], dtype=float32)

In [34]:
y_data[0]

array([0.24513718, 0.        , 0.0125504 , 0.        , 0.80040861,
       0.        , 0.67926081, 0.        , 0.        , 0.72575548,
       0.70712364, 0.62164793, 0.37142857, 0.54285714, 0.825     ])

# Inverse scaling to return in the original scale

In [49]:
y = y_data[0]
y.shape

(15,)

In [39]:
y = y.reshape(-1,1)
y.shape

(15, 1)

In [44]:
len(train_df.columns)

15

In [66]:
pd.Series(y)

0     0.245137
1     0.000000
2     0.012550
3     0.000000
4     0.800409
5     0.000000
6     0.679261
7     0.000000
8     0.000000
9     0.725755
10    0.707124
11    0.621648
12    0.371429
13    0.542857
14    0.825000
dtype: float64

In [78]:
single_test_df = pd.DataFrame(columns=train_df.columns)
single_test_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI


In [79]:
single_test_df.loc[len(test_df)] = y

In [80]:
single_test_df

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
1,0.245137,0.0,0.01255,0.0,0.800409,0.0,0.679261,0.0,0.0,0.725755,0.707124,0.621648,0.371429,0.542857,0.825


In [81]:
scaler.inverse_transform(single_test_df)

array([[6.63292192e+03, 0.00000000e+00, 2.39737802e+02, 0.00000000e+00,
        2.38081541e+04, 0.00000000e+00, 8.34281718e+04, 0.00000000e+00,
        0.00000000e+00, 2.70786851e-02, 1.27880814e-02, 6.54357481e-03,
        6.71428571e+00, 9.97142857e+00, 1.43000000e+01]])

In [35]:
# complete testing data frame

test_df_inv_normalized = pd.DataFrame(scaler.inverse_transform(test_df_normalized), columns=test_df_normalized.columns)  
test_df_inv_normalized

,UE1: web-rtc,UE1: sipp,UE1: web-server,UE2: web-rtc,UE2: sipp,UE2: web-server,UE3: web-rtc,UE3: sipp,UE3: web-server,UE1-Jitter,UE2-Jitter,UE3-Jitter,UE1-CQI,UE2-CQI,UE3-CQI
0,27058.0,0.0,0.0,0.0,26362.0,0.0,0.0,0.0,0.0,0.020393,0.016951,0.000000,13.0,15.0,15.0
1,6653.0,0.0,0.0,0.0,23589.0,0.0,0.0,0.0,0.0,0.026423,0.017882,0.000000,8.0,8.0,13.0
2,10327.0,0.0,0.0,0.0,29718.0,0.0,0.0,0.0,0.0,0.027914,0.016719,0.000000,11.0,14.0,15.0
3,10905.0,0.0,0.0,0.0,26210.0,0.0,0.0,0.0,0.0,0.028729,0.016631,0.000000,3.0,8.0,13.0
4,12492.0,0.0,0.0,0.0,26210.0,0.0,0.0,0.0,24484.0,0.028864,0.016647,0.005964,7.0,8.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0.0,0.0,0.0,13414.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017833,0.000000,7.0,8.0,13.0
113,0.0,0.0,0.0,16824.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017895,0.000000,7.0,8.0,13.0
114,0.0,0.0,0.0,6506.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017997,0.000000,8.0,8.0,12.0
115,0.0,0.0,0.0,20683.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017994,0.000000,7.0,8.0,12.0


# Predictions - Evaluation

## Training Evaluation

We load the (same) model from the saved HDF5 file in our repository. We begin by evaluating the model on the training - augmented data.

In [84]:
from keras.models import load_model

In [85]:
model = load_model('/content/gdrive/MyDrive/app_aware_CNN-LSTM_15cols.best.hdf5')

In [86]:
X_data.shape

(35066, 30, 15)

In [87]:
yhat_training = model.predict(X_data)

In [88]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

rms = mean_squared_error(y_data, yhat_training)
print('RMSE training',rms)

mae = mean_absolute_error(y_data, yhat_training)
print('MAE: training',mae)

RMSE training 0.0006376426034160214
MAE: training 0.014363775232536248


In our algorithm we use the *round()* function to round the predictions.

In [90]:
print(yhat_training[100])
print(y_data[100])

[ 4.9214852e-01 -4.2162656e-06  9.2478003e-04  3.5791993e-03
  7.9616851e-01  6.5679287e-06  6.1696153e-02 -7.1523630e-04
 -3.6110207e-03  6.8075377e-01  6.6338158e-01  2.7140358e-01
  4.5125139e-01  5.0097775e-01  6.5513259e-01]
[0.50690723 0.         0.         0.         0.78081308 0.
 0.0820753  0.         0.         0.73302168 0.67818086 0.24588098
 0.48571429 0.48571429 0.825     ]


In [92]:
for i in yhat_training[100]:
  print(round(i,2))

0.49
-0.0
0.0
0.0
0.8
0.0
0.06
-0.0
-0.0
0.68
0.66
0.27
0.45
0.5
0.66


## Testing Evaluation

Now, use the validation X data (X-val) from the basic non-augmented scenario to make predictions. Then evaluate the model utilizing the MAE, and RMSE metrics. 

In [93]:
yhat_testing = model.predict(X_val)

In [94]:
rms = mean_squared_error(y_val, yhat_testing)
print('RMSE testing ',rms)

mae = mean_absolute_error(y_val, yhat_testing)
print('MAE: testing ',mae)

RMSE testing  0.01467135765843876
MAE: testing  0.07173319491322112
